In [1]:
import argparse
from copy import deepcopy
from typing import Tuple, Union, List

import numpy as np
from batchgenerators.augmentations.utils import resize_segmentation
from nnunet.inference.segmentation_export import save_segmentation_nifti_from_softmax, save_segmentation_nifti
from batchgenerators.utilities.file_and_folder_operations import *
from multiprocessing import Process, Queue
import torch
import SimpleITK as sitk
import shutil
from multiprocessing import Pool
from nnunet.postprocessing.connected_components import load_remove_save, load_postprocessing
from nnunet.training.model_restore import load_model_and_checkpoint_files
from nnunet.training.network_training.nnUNetTrainer import nnUNetTrainer
from nnunet.utilities.one_hot_encoding import to_one_hot


Please cite the following paper when using nnUNet:
Fabian Isensee, Paul F. Jäger, Simon A. A. Kohl, Jens Petersen, Klaus H. Maier-Hein "Automated Design of Deep Learning Methods for Biomedical Image Segmentation" arXiv preprint arXiv:1904.08128 (2020).
If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [13]:
path_sample_pickle_file_3d_fullres = "/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_trained_models/nnUNet/3d_cascade_fullres/Task003_Liver/nnUNetTrainerV2CascadeFullRes__nnUNetPlansv2.1/plans.pkl"

In [14]:
def load_pickle(file, mode='rb'):
    with open(file, mode) as f:
        a = pickle.load(f)
    return a

In [15]:
num_modalities = load_pickle(path_sample_pickle_file_3d_fullres)['num_modalities']

In [16]:
num_modalities

1

In [20]:
input_folder = "/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/"

In [21]:
def check_input_folder_and_return_caseIDs(input_folder, expected_num_modalities):
    print("This model expects %d input modalities for each image" % expected_num_modalities)
    files = subfiles(input_folder, suffix=".nii.gz", join=False, sort=True)

    maybe_case_ids = np.unique([i[:-12] for i in files])

    remaining = deepcopy(files)
    missing = []

    assert len(files) > 0, "input folder did not contain any images (expected to find .nii.gz file endings)"

    # now check if all required files are present and that no unexpected files are remaining
    for c in maybe_case_ids:
        for n in range(expected_num_modalities):
            expected_output_file = c + "_%04.0d.nii.gz" % n
            if not isfile(join(input_folder, expected_output_file)):
                missing.append(expected_output_file)
            else:
                remaining.remove(expected_output_file)

    print("Found %d unique case ids, here are some examples:" % len(maybe_case_ids),
          np.random.choice(maybe_case_ids, min(len(maybe_case_ids), 10)))
    print("If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc")

    if len(remaining) > 0:
        print("found %d unexpected remaining files in the folder. Here are some examples:" % len(remaining),
              np.random.choice(remaining, min(len(remaining), 10)))

    if len(missing) > 0:
        print("Some files are missing:")
        print(missing)
        raise RuntimeError("missing files in input_folder")

    return maybe_case_ids

In [22]:
case_ids = check_input_folder_and_return_caseIDs(input_folder, 1)

This model expects 1 input modalities for each image
Found 5 unique case ids, here are some examples: ['liver_69' 'liver_18' 'liver_77' 'liver_120' 'liver_14']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc


In [23]:
output_folder = "/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output"

In [24]:
output_files = [join(output_folder, i + ".nii.gz") for i in case_ids]

In [25]:
output_files

['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output/liver_120.nii.gz',
 '/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output/liver_14.nii.gz',
 '/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output/liver_18.nii.gz',
 '/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output/liver_69.nii.gz',
 '/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest_output/liver_77.nii.gz']

In [26]:
all_files = subfiles(input_folder, suffix=".nii.gz", join=False, sort=True)
all_files

['liver_120_0000.nii.gz',
 'liver_14_0000.nii.gz',
 'liver_18_0000.nii.gz',
 'liver_69_0000.nii.gz',
 'liver_77_0000.nii.gz']

In [27]:
list_of_lists = [[join(input_folder, i) for i in all_files if i[:len(j)].startswith(j) and
                      len(i) == (len(j) + 12)] for j in case_ids]
list_of_lists


[['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/liver_120_0000.nii.gz'],
 ['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/liver_14_0000.nii.gz'],
 ['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/liver_18_0000.nii.gz'],
 ['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/liver_69_0000.nii.gz'],
 ['/net/pasnas01/pool1/ramanha/corpus/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task011_LiverTest/imagesTr/liver_77_0000.nii.gz']]